In [1]:
from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date
import numpy as np
import pandas as pd
import pandas_datareader.data as web
from pandas import DataFrame
import pickle

## Black-Scholes Model
Implementation of the Black-Scholes Model for the price calculation of european Call/Put options.
<br>
<br>
__Symbols:__
<br>
S = Current Stock Price
<br>
K = Strike Price
<br>
t = time to maturity
<br>
r = risk-free rate (US Treasury Bonds, because of missing API support)
<br>
sigma = volatility

In [29]:
# calculation
def d1(S,K,t,r,sigma):
    return (1/sigma*sqrt(t))*(log(S/K)+(r+(sigma**2)/2)*t)
def d2(S,K,t,r,sigma):
    return d1(S,K,t,r,sigma)-sigma*sqrt(t)

In [30]:
# european call
def bs_call(S,K,T,r,sigma):
    return S*norm.cdf(d1(S,K,T,r,sigma))-K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma))

# european put
def bs_put(S,K,T,r,sigma):
    return norm.cdf(-d2(S,K,T,r,sigma))*K*exp(-r*T)-norm.cdf(-d1(S,K,T,r,sigma))*S

## Getting Data via Alpha Vantage

In [15]:
today = date.today()
#print(today)
one_year_ago = today.replace(year=today.year-1)
print("today: " + str(today.day))
print("last year: " + str(one_year_ago))

today: 23
last year: 2021-05-23


### getting company data

In [7]:
av_key = "C8QYMTKO2IMFJD27"

# just 4 companys due to API restrictions
dax_40 = {
    "adidas" : web.DataReader("ADS.DE", 'av-daily', one_year_ago, today, api_key = av_key),
    "airbus" : web.DataReader("AIR.DE", 'av-daily', one_year_ago, today, api_key = av_key),
    "allianz" : web.DataReader("ALV.DE", 'av-daily', one_year_ago, today, api_key = av_key),
    "basf" : web.DataReader("BAS.DE", 'av-daily', one_year_ago, today, api_key = av_key),
}
dax_40["adidas"]

,open,high,low,close,volume
2021-05-25,304.40,304.40,299.15,299.15,555295
2021-05-26,299.00,299.35,294.45,297.05,515046
2021-05-27,295.50,298.35,293.35,297.25,578908
2021-05-28,298.60,299.90,295.10,299.35,315651
2021-05-31,299.00,302.90,297.65,298.35,279491
...,...,...,...,...,...
2022-05-16,179.46,181.68,177.82,180.06,648134
2022-05-17,181.90,186.82,180.44,183.30,630018
2022-05-18,183.40,184.08,175.14,175.98,782072
2022-05-19,175.98,175.98,175.98,175.98,730987


In [9]:
for k in dax_40.keys():
    dax_40[k] = dax_40[k].sort_index(ascending = False)
    dax_40[k] = dax_40[k].dropna()
    dax_40[k] = dax_40[k].assign(close_day_before=dax_40[k].close.shift(-1))
    dax_40[k]['returns'] = ((dax_40[k].close - dax_40[k].close_day_before)/dax_40[k].close_day_before)
dax_40["adidas"]

,open,high,low,close,volume,close_day_before,returns
2022-05-20,170.24,175.34,168.30,172.80,1035957,175.98,-0.018070
2022-05-19,175.98,175.98,175.98,175.98,730987,175.98,0.000000
2022-05-18,183.40,184.08,175.14,175.98,782072,183.30,-0.039935
2022-05-17,181.90,186.82,180.44,183.30,630018,180.06,0.017994
2022-05-16,179.46,181.68,177.82,180.06,648134,180.48,-0.002327
...,...,...,...,...,...,...,...
2021-06-01,300.80,301.00,296.65,298.00,456486,298.35,-0.001173
2021-05-31,299.00,302.90,297.65,298.35,279491,299.35,-0.003341
2021-05-28,298.60,299.90,295.10,299.35,315651,297.25,0.007065
2021-05-27,295.50,298.35,293.35,297.25,578908,297.05,0.000673


## Calculating Option Price

### setting up parameters for Calculation

In [31]:
uty = (web.DataReader(
    "^TNX", 'yahoo', today.replace(day=today.day-3), today)['Close'].iloc[-1])/100

In [36]:
stock = 'adidas' # full company name
expiry = '2022-09-14'
strike_price = 200
relation = 0.1 #Bezugsverhältnis

In [37]:
# just execute, no changes
df = dax_40[stock]

sigma = np.sqrt(252) * df['returns'].std()
lcp = df['close'].iloc[-1]
t = (datetime.strptime(expiry, "%Y-%m-%d") - datetime.utcnow()).days / 365


print("+++ " + str(stock) + " +++")
print('The Put Option Price is: ', bs_put(lcp, strike_price, t, uty, sigma)*relation)
print('The Call Option Price is: ', bs_call(lcp, strike_price, t, uty, sigma)*relation)

+++ adidas +++
The Put Option Price is:  -1.1633448374192383
The Call Option Price is:  8.713477760604254
